In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from shapely.geometry import Point
from shapely.geometry import shape
import geopandas as gpd
from osgeo import gdal
import numpy as np
import os
import requests

In [2]:
cdl_codes = pd.read_csv("../unsynced-data/cdl-codes.csv")

In [3]:
#Load Data
counties = gpd.read_file("../unsynced-data/county-crops-v1.geojson")
counties = counties.to_crs("EPSG:4326")

In [49]:
cdl_codes = cdl_codes[cdl_codes.Class_Names != ' ']
cdl_codes = cdl_codes.reset_index(drop=True)
cdl_codes


,Codes,crop_code,Class_Names,category,Erdas_Red,Erdas_Green,Erdas_Blue,ESRI_Red,ESRI_Green,ESRI_Blue
0,0,crop_0,Background,meta,0.000,0.0000,0.0000,0,0,0
1,1,crop_1,Corn,food,1.000,0.8275,0.0000,255,212,0
2,2,crop_2,Cotton,commodity,1.000,0.1490,0.1490,255,38,38
3,3,crop_3,Rice,food,0.000,0.6588,0.8980,0,169,230
4,4,crop_4,Sorghum,food,1.000,0.6196,0.0471,255,158,15
...,...,...,...,...,...,...,...,...,...,...
129,247,crop_247,Turnips,food,1.000,0.4000,0.4000,255,102,102
130,248,crop_248,Eggplants,food,1.000,0.4000,0.4000,255,102,102
131,249,crop_249,Gourds,food,1.000,0.4000,0.4000,255,102,102
132,250,crop_250,Cranberries,food,1.000,0.4000,0.4000,255,102,102


In [19]:
def c_to_crop(code):
    return cdl_codes[cdl_codes["Codes"] == code]["Class_Names"].values[0]
def cc_to_crop(crop_code):
    return cdl_codes[cdl_codes["crop_code"] == crop_code]["Class_Names"].values[0]

In [ ]:
col_list = []
for c in counties.columns:
    print(c)
    col_list.append(c)
crop_col = []
for c in col_list:
    if c.startswith("crop"):
        crop_col.append(c)

In [29]:
statefp = pd.read_csv("../unsynced-data/statefpcodes.csv")
# join State column and Postal column to counties based on FIPS and STATEFP
counties = counties.merge(statefp, left_on="STATEFP", right_on="FIPS")
counties = counties.drop(columns=["Unnamed: 3", "Unnamed: 4", "Unnamed: 5", "FIPS"])

In [37]:
counties["total_crop"] = counties[crop_col].sum(axis=1)
counties["max_crop_all"] = counties[crop_col].max(axis=1)

In [ ]:
# create a python list of all crop_codes that match category food in cdl_codes
food_crop = []
for i in range(len(cdl_codes)):
    if cdl_codes["category"][i] == "food":
        food_crop.append(cdl_codes["crop_code"][i])
# food_crop = [x for x in food_crop if x not in ['crop_210', 'crop_215', 'crop_228', 'crop_230', 'crop_234', 'crop_235']]
# counties["total_crop_food"] = counties[food_crop].sum(axis=1)
# counties["max_crop_food"] = counties[food_crop].max(axis=1)

In [97]:
counties.to_file("../unsynced-data/county-crops-v2.geojson", driver="GeoJSON")

In [93]:
counties

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,...,SOYBEANS_unit,SUGARBEETS_unit,SUGARCANE_unit,SUNFLOWER_unit,SWITCHGRASS_unit,TOBACCO_unit,TRITICALE_unit,WHEAT_unit,WILD RICE_unit,SWEET CORN_unit
0,31,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,None,...,0,0,0,0,0,0,0,0,0,0
1,31,109,00835876,31109,Lancaster,Lancaster County,06,H1,G4020,339,...,0,0,0,0,0,0,0,0,0,0
2,31,129,00835886,31129,Nuckolls,Nuckolls County,06,H1,G4020,None,...,0,0,0,0,0,0,0,0,0,0
3,31,101,00835872,31101,Keith,Keith County,06,H1,G4020,None,...,0,0,0,0,0,0,0,0,0,0
4,31,137,00835890,31137,Phelps,Phelps County,06,H1,G4020,None,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2902,44,009,01219782,44009,Washington,Washington County,06,H4,G4020,148,...,0,0,0,0,0,0,0,0,0,0
2903,44,007,01219781,44007,Providence,Providence County,06,H4,G4020,148,...,0,0,0,0,0,0,0,0,0,0
2904,44,001,01219777,44001,Bristol,Bristol County,06,H4,G4020,148,...,0,0,0,0,0,0,0,0,0,0
2905,44,005,01219779,44005,Newport,Newport County,06,H4,G4020,148,...,0,0,0,0,0,0,0,0,0,0


In [67]:
kcal_conversion = pd.read_csv("../unsynced-data/Stability_Crop_Diversity-2.0/Data/Inputs/kcal_conversion.csv")
kcal_conversion

,Crop,Crop_Class,Crop_Utilization,Crop_Production,Production_Units,Conversion_to_kg,kcal_kg
0,CAULIFLOWER,ALL CLASSES,ALL UTILIZATION PRACTICES,ALL PRODUCTION PRACTICES,CWT,45.35920,98.0
1,ASPARAGUS,ALL CLASSES,ALL UTILIZATION PRACTICES,ALL PRODUCTION PRACTICES,CWT,45.35920,106.0
2,LETTUCE,HEAD,FRESH MARKET,ALL PRODUCTION PRACTICES,CWT,45.35920,114.0
3,LETTUCE,HEAD,ALL UTILIZATION PRACTICES,ALL PRODUCTION PRACTICES,CWT,45.35920,114.0
4,LETTUCE,LEAF,FRESH MARKET,ALL PRODUCTION PRACTICES,CWT,45.35920,140.0
...,...,...,...,...,...,...,...
109,COTTON,UPLAND,ALL UTILIZATION PRACTICES,ALL PRODUCTION PRACTICES,480 LB BALES,217.68480,NaN
110,HAY,ALL CLASSES,ALL UTILIZATION PRACTICES,ALL PRODUCTION PRACTICES,TONS,907.18500,NaN
111,HOPS,ALL CLASSES,ALL UTILIZATION PRACTICES,ALL PRODUCTION PRACTICES,LB,0.45351,NaN
112,SORGHUM,ALL CLASSES,SILAGE,ALL PRODUCTION PRACTICES,TONS,907.18500,NaN


Made this request and downloaded all crop production data by county:

https://quickstats.nass.usda.gov/api/api_GET/?key=A5DB238B-8B04-3675-8A54-F373FA342D6F&source_desc=CENSUS&sector_desc=CROPS&agg_level_desc=COUNTY&domain_desc=TOTAL&statisticcat_desc=PRODUCTION&year__GE=2017&format=CSV

In [80]:
# all crop data by county in 2017
usda = pd.read_csv("../unsynced-data/usda_allcrops_county_2017.csv")
usda

,group_desc,commodity_desc,class_desc,prodn_practice_desc,util_practice_desc,unit_desc,short_desc,state_ansi,state_fips_code,state_alpha,state_name,asd_code,asd_desc,county_ansi,county_code,county_name,location_desc,Value,CV (%)
0,CROP TOTALS,CROP TOTALS,ALL CLASSES,PRODUCTION CONTRACT,ALL UTILIZATION PRACTICES,OPERATIONS,"CROP TOTALS, PRODUCTION CONTRACT - OPERATIONS ...",1,1,AL,ALABAMA,60,WIREGRASS,69.0,69,HOUSTON,"ALABAMA, WIREGRASS, HOUSTON",1,0.4
1,CROP TOTALS,CROP TOTALS,ALL CLASSES,PRODUCTION CONTRACT,ALL UTILIZATION PRACTICES,OPERATIONS,"CROP TOTALS, PRODUCTION CONTRACT - OPERATIONS ...",4,4,AZ,ARIZONA,80,SOUTHERN,27.0,27,YUMA,"ARIZONA, SOUTHERN, YUMA",4,82.1
2,CROP TOTALS,CROP TOTALS,ALL CLASSES,PRODUCTION CONTRACT,ALL UTILIZATION PRACTICES,OPERATIONS,"CROP TOTALS, PRODUCTION CONTRACT - OPERATIONS ...",5,5,AR,ARKANSAS,60,EAST CENTRAL,85.0,85,LONOKE,"ARKANSAS, EAST CENTRAL, LONOKE",1,0.2
3,CROP TOTALS,CROP TOTALS,ALL CLASSES,PRODUCTION CONTRACT,ALL UTILIZATION PRACTICES,OPERATIONS,"CROP TOTALS, PRODUCTION CONTRACT - OPERATIONS ...",6,6,CA,CALIFORNIA,40,CENTRAL COAST,79.0,79,SAN LUIS OBISPO,"CALIFORNIA, CENTRAL COAST, SAN LUIS OBISPO",1,(L)
4,CROP TOTALS,CROP TOTALS,ALL CLASSES,PRODUCTION CONTRACT,ALL UTILIZATION PRACTICES,OPERATIONS,"CROP TOTALS, PRODUCTION CONTRACT - OPERATIONS ...",6,6,CA,CALIFORNIA,80,SOUTHERN CALIFORNIA,65.0,65,RIVERSIDE,"CALIFORNIA, SOUTHERN CALIFORNIA, RIVERSIDE",1,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46558,VEGETABLES,SWEET CORN,ALL CLASSES,ALL PRODUCTION PRACTICES,SEED,LB,"SWEET CORN, SEED - PRODUCTION, MEASURED IN LB",53,53,WA,WASHINGTON,50,EAST CENTRAL,1.0,1,ADAMS,"WASHINGTON, EAST CENTRAL, ADAMS","735,000",8.9
46559,VEGETABLES,SWEET CORN,ALL CLASSES,ALL PRODUCTION PRACTICES,SEED,LB,"SWEET CORN, SEED - PRODUCTION, MEASURED IN LB",53,53,WA,WASHINGTON,50,EAST CENTRAL,21.0,21,FRANKLIN,"WASHINGTON, EAST CENTRAL, FRANKLIN",(D),(D)
46560,VEGETABLES,SWEET CORN,ALL CLASSES,ALL PRODUCTION PRACTICES,SEED,LB,"SWEET CORN, SEED - PRODUCTION, MEASURED IN LB",53,53,WA,WASHINGTON,50,EAST CENTRAL,25.0,25,GRANT,"WASHINGTON, EAST CENTRAL, GRANT","318,000",8.9
46561,VEGETABLES,SWEET CORN,ALL CLASSES,ALL PRODUCTION PRACTICES,SEED,LB,"SWEET CORN, SEED - PRODUCTION, MEASURED IN LB",55,55,WI,WISCONSIN,50,CENTRAL,1.0,1,ADAMS,"WISCONSIN, CENTRAL, ADAMS",(D),(D)


In [88]:
# how many unique values in each column of usda
for c in usda.columns:
    print(c, len(usda[c].unique()))
    

group_desc 3
commodity_desc 48
class_desc 38
prodn_practice_desc 2
util_practice_desc 8
unit_desc 8
short_desc 87
state_ansi 50
state_fips_code 50
state_alpha 50
state_name 50
asd_code 23
asd_desc 86
county_ansi 267
county_code 269
county_name 1774
location_desc 3056
Value 24967
CV (%) 569


In [83]:
counties.columns

Index(['STATEFP', 'COUNTYFP', 'COUNTYNS', 'GEOID', 'NAME', 'NAMELSAD', 'LSAD',
       'CLASSFP', 'MTFCC', 'CSAFP',
       ...
       'crop_249', 'crop_250', 'crop_254', 'geometry', 'State', 'Postal',
       'total_crop', 'max_crop_all', 'total_crop_food', 'max_crop_food'],
      dtype='object', length=144)

In [84]:
# create a new column in counties for each unique value in the commodity_desc column
for i in range(len(usda["commodity_desc"].unique())):
    counties[usda["commodity_desc"].unique()[i]] = 0


In [86]:
counties.columns

Index(['STATEFP', 'COUNTYFP', 'COUNTYNS', 'GEOID', 'NAME', 'NAMELSAD', 'LSAD',
       'CLASSFP', 'MTFCC', 'CSAFP',
       ...
       'SOYBEANS', 'SUGARBEETS', 'SUGARCANE', 'SUNFLOWER', 'SWITCHGRASS',
       'TOBACCO', 'TRITICALE', 'WHEAT', 'WILD RICE', 'SWEET CORN'],
      dtype='object', length=192)

dummy check. columns went from 144 to 192. This lines up with the 48 unique values 

In [90]:
for i in range(len(usda["commodity_desc"].unique())):
    counties[usda["commodity_desc"].unique()[i]+"_unit"] = 0

In [ ]:
# for each of the rows in usda add the value of the value column to the column in counties that matches the commodity_desc in the row where usda county_code and counties COUNTYFP match
for i in range(len(usda)):
    if usda["county_code"][i] == counties["COUNTYFP"][i]:
        counties[usda["commodity_desc"][i]][i] = usda["value"][i]        

# Going with State Level Approach

1. Create DF with county crop pixels "unsynced-data/county-crops-v1.geojson"
2. Create a conversion dataframe with total pixels of each crop across the US with Crop Name
3. Use state level data to convert pixels to kCals "unsynced-data/Stability_Crop_Diversity-2.0/Data/Outputs/Intermediate_Data/Clean_Data.csv"
4. for every column in county level crops create a column with the kCals of that crop

In [4]:
counties = gpd.read_file("../unsynced-data/county-crops-v1.geojson")
counties = counties.to_crs("EPSG:4326")
for c in counties.columns:
    if c.startswith("crop"):
        counties[c] = counties[c].replace(0, np.nan)

production = pd.read_csv("../unsynced-data/Stability_Crop_Diversity-2.0/Data/Outputs/Intermediate_Data/Clean_Data.csv")
production = production.drop(columns=["Unnamed: 0"])

statefp = pd.read_csv("../unsynced-data/statefpcodes.csv")
counties = counties.merge(statefp, left_on="STATEFP", right_on="FIPS")
counties = counties.drop(columns=["Unnamed: 3", "Unnamed: 4", "Unnamed: 5", "FIPS"])

final_crops = pd.read_csv("../unsynced-data/final_crops.csv")

In [ ]:
# creates a dataframe with each unique crop name and the data associated with a row that has that crop name
# this was used to make the final_crops.csv file that will be all the crops we use in the final data
unique = production['Crop_Name'].unique().tolist()
unique_df = pd.DataFrame()
for u in unique:
    unique_df = unique_df.append(production[production['Crop_Name'] == u].iloc[0])
# write unique_df to csv
# unique_df.to_csv("../unsynced-data/Production_data_clean.csv")

Developing a function that will add kcal's to counties df
1. step 1 rollup counties data when needed as shown in final_crops dataframe
2. delete all columns that aren't present in the final_crops data
3. choose the closest year to 2017 or most recent in the production data
4. roll up values in production data when needed as shown in final_crops dataframe

In [12]:
for c in counties.columns:
    if c.startswith("crop"):
        counties[c] = counties[c].replace(0, np.nan)

In [13]:
counties

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,...,crop_245,crop_246,crop_247,crop_248,crop_249,crop_250,crop_254,geometry,State,Postal
0,31,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-96.55515 41.91587, -96.55515 ...",Nebraska,NE
1,31,109,00835876,31109,Lancaster,Lancaster County,06,H1,G4020,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-96.68140 41.04566, -96.68139 ...",Nebraska,NE
2,31,129,00835886,31129,Nuckolls,Nuckolls County,06,H1,G4020,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-98.04802 40.35066, -98.04674 ...",Nebraska,NE
3,31,101,00835872,31101,Keith,Keith County,06,H1,G4020,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-101.73016 41.39521, -101.7263...",Nebraska,NE
4,31,137,00835890,31137,Phelps,Phelps County,06,H1,G4020,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-99.17949 40.52501, -99.17948 ...",Nebraska,NE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2902,44,009,01219782,44009,Washington,Washington County,06,H4,G4020,148,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-71.57520 41.32094, -71.57525 ...",Rhode Island,RI
2903,44,007,01219781,44007,Providence,Providence County,06,H4,G4020,148,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-71.54735 41.73120, -71.54743 ...",Rhode Island,RI
2904,44,001,01219777,44001,Bristol,Bristol County,06,H4,G4020,148,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-71.21043 41.68801, -71.21086 ...",Rhode Island,RI
2905,44,005,01219779,44005,Newport,Newport County,06,H4,G4020,148,...,NaN,NaN,NaN,NaN,NaN,32.0,NaN,"MULTIPOLYGON (((-71.11640 41.48457, -71.11543 ...",Rhode Island,RI


In [22]:
production

,State_Abbr,Year,Crop_Name,Price_Received_USD_kg,Crop_Area_ha,Production_kg,FIPS,kcal_kg,Production_kcal,Production_USD
0,CA,1998,TOMATOES-ALL CLASSES,0.106907,130716.309187,8.535522e+09,6,187.0,1.596143e+12,1.317294e+09
1,CA,1999,CARROTS-ALL CLASSES,0.343009,30109.267503,1.029173e+09,6,365.0,3.756482e+11,5.417210e+08
2,CA,1999,TOMATOES-ALL CLASSES,0.097394,152165.115338,1.165760e+10,6,187.0,2.179971e+12,1.742295e+09
3,CA,2000,CARROTS-ALL CLASSES,0.266750,29906.920275,1.025345e+09,6,365.0,3.742509e+11,4.150759e+08
4,CA,2000,TOMATOES-ALL CLASSES,0.099191,127478.753541,9.910542e+09,6,187.0,1.853271e+12,1.491845e+09
...,...,...,...,...,...,...,...,...,...,...
22532,WY,2020,BARLEY-ALL CLASSES,0.226433,27114.528531,1.400401e+08,56,3530.0,4.943415e+11,3.032409e+07
22533,WY,2020,CORN-ALL CLASSES-GRAIN,0.155504,21853.500607,1.673431e+08,56,3650.0,6.108023e+11,2.488545e+07
22534,WY,2020,HAY-ALL CLASSES,0.188495,437070.012141,2.440328e+09,56,NaN,NaN,4.398891e+08
22535,WY,2020,SUGARBEETS-ALL CLASSES,0.052690,12383.650344,8.219096e+08,56,700.0,5.753367e+11,4.141436e+07


In [47]:
final_crops

,crop_code,cdl_name,production_name,final_name,kcal_kg,red,green,blue,rollup
0,crop_75,Almonds,ALMONDS-ALL CLASSES,Almonds,5790.0,0,168,132,NaN
1,crop_68,Apples,APPLES-ALL CLASSES,Apples,468.0,185,0,80,NaN
2,crop_223,Apricots,APRICOTS-ALL CLASSES,Apricots,446.0,255,145,171,NaN
3,crop_207,Asparagus,ASPARAGUS-ALL CLASSES,Asparagus,106.0,255,102,102,NaN
4,crop_215,Avocados,AVOCADOS-ALL CLASSES,Avocados,1184.0,102,153,77,NaN
5,crop_21,Barley,BARLEY-ALL CLASSES,Barley,3530.0,226,0,127,NaN
6,crop_42,Dry Beans,"BEANS-DRY EDIBLE, INCL CHICKPEAS",Dry Beans,3506.0,168,0,0,crop_42; crop_51
7,crop_242,Blueberries,BLUEBERRIES-TAME,Blueberries,542.0,0,0,153,NaN
8,crop_214,Broccoli,BROCCOLI-ALL CLASSES,Broccoli,207.0,255,102,102,NaN
9,crop_243,Cabbage,CABBAGE-ALL CLASSES,Cabbage,200.0,255,102,102,NaN


In [6]:
# rolling up counties data as dictated by final crops
counties["crop_42"] = counties["crop_42"] + counties["crop_51"]
counties["crop_22"] = counties["crop_22"] + counties["crop_23"] + counties["crop_24"]
counties = counties.drop(columns=["crop_51", "crop_23", "crop_24"])

In [ ]:
# rolling up production data as dictated by final crops
# this basically takes each crop from each state and adds the row that is closest to 2017 to the rel_prod dataframe
rel_prod = pd.DataFrame()
for s in production["State_Abbr"].unique():
    state = production[production["State_Abbr"] == s]
    for c in state["Crop_Name"].unique():
        crop = state[state["Crop_Name"] == c]
        rel_prod = rel_prod.append(crop.iloc[(crop['Year']-2017).abs().argsort()[:1]])

In [ ]:
# joining rollup rows on production data
roll1 = ["LETTUCE-HEAD", "LETTUCE-LEAF", "LETTUCE-ROMAINE"]
roll1_df = pd.DataFrame()
roll1_df = roll1_df.append(rel_prod.iloc[0])
roll1_df = roll1_df.drop(roll1_df.index[0])
roll1_df
for s in rel_prod["State_Abbr"].unique():
    state = rel_prod[rel_prod["State_Abbr"] == s]
    # make a data frame of values that match roll1 and state
    temp = state[state["Crop_Name"].isin(roll1)]
    # add a row to roll1_df that includes state abbreviation, most recent year, "LETTUCE-HEAD", average Price_Received_USD_kg, sum of values in in Crop_Area_ha, sum of values in Production_kg, FIPS in FIPS column, 138 kcal_kg in kcal_kg column, sum of values in Production_kcal, sum of values in Production_USD columns
    # if temp length is longer than 0
    if len(temp) > 0:
        roll1_df = roll1_df.append({
            "State_Abbr": s, "Year": temp["Year"].max(), 
            "Crop_Name": "LETTUCE-HEAD", 
            "Price_Received_USD_kg": temp["Price_Received_USD_kg"].mean(), 
            "Crop_Area_ha": temp["Crop_Area_ha"].sum(), "Production_kg": temp["Production_kg"].sum(), 
            "FIPS": temp["FIPS"].iloc[0], 
            "kcal_kg": temp["kcal_kg"].mean(),
            "Production_kcal": temp["Production_kcal"].sum(), 
            "Production_USD": temp["Production_USD"].sum()
            }, ignore_index=True)
roll1_df = roll1_df.drop_duplicates(subset="State_Abbr", keep="first")
for i in range(len(roll1)):
    rel_prod = rel_prod[rel_prod["Crop_Name"] != roll1[i]]
rel_prod = rel_prod.append(roll1_df)

roll2 = ["PEAS-GREEN", "PEAS-DRY EDIBLE", "PEAS-AUSTRIAN WINTER"]
roll2_df = pd.DataFrame()
roll2_df = roll2_df.append(rel_prod.iloc[0])
roll2_df = roll2_df.drop(roll2_df.index[0])
for s in rel_prod["State_Abbr"].unique():
    state = rel_prod[rel_prod["State_Abbr"] == s]
    temp = state[state["Crop_Name"].isin(roll2)]
    if len(temp) > 0:
        roll2_df = roll2_df.append({
            "State_Abbr": s, "Year": temp["Year"].max(), 
            "Crop_Name": "PEAS-GREEN", 
            "Price_Received_USD_kg": temp["Price_Received_USD_kg"].mean(), 
            "Crop_Area_ha": temp["Crop_Area_ha"].sum(), "Production_kg": temp["Production_kg"].sum(), 
            "FIPS": temp["FIPS"].iloc[0], 
            "kcal_kg": temp["kcal_kg"].mean(), 
            "Production_kcal": temp["Production_kcal"].sum(), 
            "Production_USD": temp["Production_USD"].sum()
            }, ignore_index=True)
roll2_df = roll2_df.drop_duplicates(subset="State_Abbr", keep="first")
for i in range(len(roll2)):
    rel_prod = rel_prod[rel_prod["Crop_Name"] != roll2[i]]
rel_prod = rel_prod.append(roll2_df)

roll3 = ["GRAPEFRUIT-ALL CLASSES", "LEMONS-ALL CLASSES", "TANGELOS-ALL CLASSES", "TANGERINES-ALL CLASSES"]
roll3_df = pd.DataFrame()
roll3_df = roll3_df.append(rel_prod.iloc[0])
roll3_df = roll3_df.drop(roll3_df.index[0])
for s in rel_prod["State_Abbr"].unique():
    state = rel_prod[rel_prod["State_Abbr"] == s]
    temp = state[state["Crop_Name"].isin(roll3)]
    if len(temp) > 0:
        roll3_df = roll3_df.append({
            "State_Abbr": s, "Year": temp["Year"].max(), 
            "Crop_Name": "GRAPEFRUIT-ALL CLASSES", 
            "Price_Received_USD_kg": temp["Price_Received_USD_kg"].mean(), 
            "Crop_Area_ha": temp["Crop_Area_ha"].sum(), "Production_kg": temp["Production_kg"].sum(), 
            "FIPS": temp["FIPS"].iloc[0], 
            "kcal_kg": temp["kcal_kg"].mean(), 
            "Production_kcal": temp["Production_kcal"].sum(), 
            "Production_USD": temp["Production_USD"].sum()
            }, ignore_index=True)
roll3_df = roll3_df.drop_duplicates(subset="State_Abbr", keep="first")
for i in range(len(roll3)):
    rel_prod = rel_prod[rel_prod["Crop_Name"] != roll3[i]]
rel_prod = rel_prod.append(roll3_df)
rel_prod = rel_prod.reset_index(drop=True)

In [11]:
rel_prod

,State_Abbr,Year,Crop_Name,Price_Received_USD_kg,Crop_Area_ha,Production_kg,FIPS,kcal_kg,Production_kcal,Production_USD
0,CA,2017,TOMATOES-ALL CLASSES,0.106924,100242.816673,1.005575e+10,6,187.000000,1.880425e+12,1.003435e+09
1,CA,2017,CARROTS-ALL CLASSES,0.648160,23674.625658,9.817999e+08,6,365.000000,3.583570e+11,5.938864e+08
2,CA,2017,BARLEY-ALL CLASSES,0.221381,11736.139215,3.156998e+07,6,3530.000000,1.114420e+11,6.522491e+06
3,CA,2017,"BEANS-DRY EDIBLE, INCL CHICKPEAS",1.349230,20113.314448,4.740036e+07,6,3506.000000,1.661857e+11,5.968513e+07
4,CA,2017,CORN-ALL CLASSES-GRAIN,0.165347,32375.556455,3.393600e+08,6,3650.000000,1.238664e+12,5.236658e+07
...,...,...,...,...,...,...,...,...,...,...
918,SD,2017,PEAS-GREEN,0.246918,14164.305949,2.381358e+07,46,2620.000000,6.239158e+10,5.487515e+06
919,CA,2017,GRAPEFRUIT-ALL CLASSES,0.569895,46701.740186,1.767196e+09,6,280.666667,5.798218e+11,5.744850e+08
920,FL,2017,GRAPEFRUIT-ALL CLASSES,0.469951,18494.536625,3.858034e+08,12,314.666667,8.182288e+10,1.456472e+08
921,TX,2017,GRAPEFRUIT-ALL CLASSES,0.357700,6636.989073,1.741795e+08,48,160.000000,2.786872e+10,5.814527e+07


Developing pixel to kcal conversion function. I'm going to develop a national level that will take total national pixels adn 

In [ ]:
# in counties drop any column whose column name starts with "crop" and is not in final_crop[crop_code] column
# this is to remove any columns that are not in the final crop list
rel_counties = counties
for c in counties.columns:
    if c.startswith("crop") and c not in final_crops["crop_code"].values:
        rel_counties = rel_counties.drop(c, axis=1)
rel_counties

In [28]:
counties["crop_75"].sum()

0.0

In [26]:
for c in counties.columns:
    if c.startswith("crop"):
        final_crops.loc[final_crops["crop_code"] == c, "total_pixels"] = counties[c].sum()

In [30]:
rel_counties

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,...,crop_242,crop_243,crop_244,crop_245,crop_246,crop_248,crop_250,geometry,State,Postal
0,31,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,None,...,NaN,11.0,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-96.55515 41.91587, -96.55515 ...",Nebraska,NE
1,31,109,00835876,31109,Lancaster,Lancaster County,06,H1,G4020,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-96.68140 41.04566, -96.68139 ...",Nebraska,NE
2,31,129,00835886,31129,Nuckolls,Nuckolls County,06,H1,G4020,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-98.04802 40.35066, -98.04674 ...",Nebraska,NE
3,31,101,00835872,31101,Keith,Keith County,06,H1,G4020,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-101.73016 41.39521, -101.7263...",Nebraska,NE
4,31,137,00835890,31137,Phelps,Phelps County,06,H1,G4020,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-99.17949 40.52501, -99.17948 ...",Nebraska,NE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2902,44,009,01219782,44009,Washington,Washington County,06,H4,G4020,148,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-71.57520 41.32094, -71.57525 ...",Rhode Island,RI
2903,44,007,01219781,44007,Providence,Providence County,06,H4,G4020,148,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-71.54735 41.73120, -71.54743 ...",Rhode Island,RI
2904,44,001,01219777,44001,Bristol,Bristol County,06,H4,G4020,148,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-71.21043 41.68801, -71.21086 ...",Rhode Island,RI
2905,44,005,01219779,44005,Newport,Newport County,06,H4,G4020,148,...,NaN,NaN,NaN,NaN,NaN,NaN,32.0,"MULTIPOLYGON (((-71.11640 41.48457, -71.11543 ...",Rhode Island,RI


In [35]:
rel_prod.head(5)

,State_Abbr,Year,Crop_Name,Price_Received_USD_kg,Crop_Area_ha,Production_kg,FIPS,kcal_kg,Production_kcal,Production_USD
0,CA,2017,TOMATOES-ALL CLASSES,0.106924,100242.816673,1.005575e+10,6,187.0,1.880425e+12,1.003435e+09
1,CA,2017,CARROTS-ALL CLASSES,0.648160,23674.625658,9.817999e+08,6,365.0,3.583570e+11,5.938864e+08
2,CA,2017,BARLEY-ALL CLASSES,0.221381,11736.139215,3.156998e+07,6,3530.0,1.114420e+11,6.522491e+06
3,CA,2017,"BEANS-DRY EDIBLE, INCL CHICKPEAS",1.349230,20113.314448,4.740036e+07,6,3506.0,1.661857e+11,5.968513e+07
4,CA,2017,CORN-ALL CLASSES-GRAIN,0.165347,32375.556455,3.393600e+08,6,3650.0,1.238664e+12,5.236658e+07


In [ ]:
# for every unique value in "Crop_Name" add the values from those, and add them to the final_crops dataframe
for c in rel_prod["Crop_Name"].unique():
    final_crops.loc[final_crops["production_name"] == c, "Crop_Area_ha"] = rel_prod[rel_prod["Crop_Name"] == c]["Crop_Area_ha"].sum()
    final_crops.loc[final_crops["production_name"] == c, "Production_kg"] = rel_prod[rel_prod["Crop_Name"] == c]["Production_kg"].sum()
    final_crops.loc[final_crops["production_name"] == c, "Production_kcal"] = rel_prod[rel_prod["Crop_Name"] == c]["Production_kcal"].sum()
    final_crops.loc[final_crops["production_name"] == c, "Production_USD"] = rel_prod[rel_prod["Crop_Name"] == c]["Production_USD"].sum()
    final_crops.loc[final_crops["production_name"] == c, "Price_Received_USD_kg"] = rel_prod[rel_prod["Crop_Name"] == c]["Price_Received_USD_kg"].mean()

In [39]:
final_crops["kcal_pixel_national"] = final_crops["Production_kcal"] / final_crops["total_pixels"]

In [42]:
# write final_crops to csv
final_crops.to_csv("final_crops_with_kcal.csv")